In [1]:
import requests
import os
from tqdm import tqdm
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin, urlparse

def is_valid(url):
    """
    Проверяет, является ли url допустимым URL
    """
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

In [2]:
def get_all_images(url):
    """
    Возвращает все URL‑адреса изображений по одному `url`
    """
    soup = bs(requests.get(url).content, "html.parser")
    urls = []
    for img in tqdm(soup.find_all("img"), "Extracting images"):
        img_url = img.attrs.get("src")
        if not img_url:
            # если img не содержит атрибута src, просто пропустить
            continue
            # сделать URL абсолютным, присоединив домен к только что извлеченному URL
        img_url = urljoin(url, img_url)
        try:
            pos = img_url.index("?")
            img_url = img_url[:pos]
        except ValueError:
            pass
        # наконец, если URL действителен
        if is_valid(img_url):
            urls.append(img_url)
    return urls

In [3]:
def download(url, pathname):
    """
    Загружает файл по URL‑адресу и помещает его в папку `pathname`
    """
    # если путь не существует, сделать этот путь dir
    if not os.path.isdir(pathname):
        os.makedirs(pathname)
    # загружаем тело ответа по частям, а не сразу
    response = requests.get(url, stream=True)
    # get the total file size
    file_size = int(response.headers.get("Content-Length", 0))
    # получаем имя файла
    filename = os.path.join(pathname, url.split("/")[-1])
    # индикатор выполнения, изменение единицы измерения на байты вместо итераций (по умолчанию tqdm)
    progress = tqdm(response.iter_content(1024), f"Downloading {filename}", total=file_size, unit="B", unit_scale=True, unit_divisor=1024)
    with open(filename, "wb") as f:
        for data in progress.iterable:
            # записываем прочитанные данные, в файл
            f.write(data)
            # обновить вручную индикатор выполнения
            progress.update(len(data))

In [4]:
import tarfile

def download_then_pack(url, path) -> None:
    # получить все изображения
    imgs = get_all_images(url)
    for img in imgs:
        # для каждого изображения, загрузите его
        download(img, path)
    with tarfile.open(path+'.tar.gz', 'w:gz') as tar:
        tar.add(path, arcname=os.path.basename(path), recursive=True)

def download_groups_then_pack(urls: dict, path: str) -> None:
    if not os.path.exists(path):
        os.makedirs(path)
    for group, url in urls.items():
        group_path = os.path.join(path, group)
        os.makedirs(group_path)
        imgs = get_all_images(url)
        for img in imgs:
            download(img, group_path)
    with tarfile.open(path + ".tar.gz", "w:gz") as tar:
        tar.add(path, arcname=os.path.basename(path), recursive=True)
    print('Complete!')
        

    

In [5]:
from concurrent.futures import ThreadPoolExecutor

def download_groups_then_pack_multithreading(urls: dict, path:str) -> None:
    if not os.path.exists(path):
        os.makedirs(path)

    for group, url in urls.items():
        group_path = os.path.join(path, group)
        os.makedirs(group_path)
        imgs = get_all_images(url)
        with ThreadPoolExecutor(max_workers=14) as executor:
            for img in imgs:
                executor.submit(download, img, group_path)
                
    with tarfile.open(path + ".tar.gz", "w:gz") as tar:
        tar.add(path, arcname=os.path.basename(path), recursive=True)
    print('Complete!')


In [6]:
# Фейковый лимит 1000, чтобы забрать побольше изображений (Max = 100)
urls = { 
    "Architecture&Decoration": "https://collections.louvre.fr/en/recherche?limit=1000&typology[0]=9",
    "Arms&Armour" : "https://collections.louvre.fr/en/recherche?limit=1000&typology[0]=10",
    "ArtsOfTheBook" : "https://collections.louvre.fr/en/recherche?limit=1000&typology[0]=11",
    "Jewellery&Finery" : "https://collections.louvre.fr/en/recherche?limit=1000&typology[0]=12",
    "Drawing&Prints" : "https://collections.louvre.fr/en/recherche?limit=1000&typology%5B0%5D=13"
    }
# download_then_pack("https://collections.louvre.fr/en/recherche?limit=100&typology%5B0%5D=9", "./louvre")
# download_groups_then_pack(urls, "./louvre")
download_groups_then_pack_multithreading(urls, "./louvre")

Extracting images: 100%|██████████| 1003/1003 [00:00<00:00, 50012.92it/s]








































Extracting images: 100%|██████████| 1003/1003 [00:00<00:00, 50075.43it/s]
















Extracting images: 100%|██████████| 1003/1003 [00:00<00:00, 46960.25it/s]



































































































































































































Extracting images: 100%|██████████| 1003/1003 [00:00<00:00, 49979.05it/s]



























Extracting images: 100%|██████████| 1003/1003 [00:00<00:00, 44874.90it/s]








































































































Complete!


In [8]:
import yaml

import shutil
from pathlib import Path

import random

def make_dataset(path_to_archive : str, path_to_disk : str, train : float, val : float):
    # Защищяемся от подстав.
    if (train + val) != 100:
        raise ValueError("train + val must be equal to 100.")
    if not os.path.exists(path_to_archive) or not tarfile.is_tarfile(path_to_archive):
        raise ValueError("path_to_archive must exist.")
    
    # Читаем tar
    with tarfile.open(path_to_archive, "r:gz") as tar:
        # Получаем список файлов.
        tar_members_names = tar.getnames()
        
        # Общий префикс = корень архива
        tar_root = os.path.commonprefix(tar_members_names)
        
        # Получаем список подпапок
        subdirs = [name for name in tar_members_names if tar.getmember(name).isdir() and name != tar_root]
        groups = [os.path.relpath(name, tar_root) for name in subdirs]
        # Увы, но держим файл открытым, создаём папку (если нету) или пересоздаём.
        if os.path.exists(path_to_disk):
            shutil.rmtree(path_to_disk)
        os.makedirs(path_to_disk, exist_ok=True)

        # Собираем data.yaml по содержимому архива.
        yaml_data = dict({
            'path' : path_to_disk,
            "train" : "images/train",
            "val": "images/val"
        })

        # названия классов - названия подпапок из архива.
        names = dict()
        for i in range(len(groups)):
            names[i] = groups[i]
        yaml_data['names'] = names

        # записываем наш data.yaml
        with open(os.path.join(path_to_disk, "data.yaml"), 'w') as outfile:
            yaml.dump(yaml_data, outfile, default_flow_style=False, sort_keys=False)

        # генерируем наш датасет.
        for i in range(len(subdirs)):
            subdir = subdirs[i]
            # group = groups[i]
            group_root_path = Path(subdir)
            group_members_names = [name for name in tar_members_names if Path(name).parent == group_root_path]

            # destination_subdir = os.path.join(path_to_disk, group)

            random.shuffle(group_members_names)

            train_group_data = group_members_names[:int((len(group_members_names)+1) * (train / 100))]
            val_group_data = group_members_names[int((len(group_members_names)+1) * (train / 100)):]
            
            for train_data in train_group_data:
                file_name = os.path.basename(train_data)
                train_images_subdir = os.path.join(path_to_disk, "images/train")
                os.makedirs(train_images_subdir, exist_ok=True)
                tar.extract(train_data, os.path.join(train_images_subdir, file_name))

                train_labes_subdir = os.path.join(path_to_disk, "labels")
                os.makedirs(train_labes_subdir, exist_ok=True)
                with open(os.path.join(train_labes_subdir, file_name) + '.txt', 'w') as label_file:
                    label_file.write(str(i) + " 0.0 0.0 1.0 1.0")
            
            for val_data in val_group_data:
                file_name = os.path.basename(val_data)
                val_images_subdir = os.path.join(path_to_disk, "images/val")
                os.makedirs(val_images_subdir, exist_ok=True)
                tar.extract(train_data, os.path.join(val_images_subdir, file_name))
                val_labels_subdir = os.path.join(path_to_disk, "labels")
                os.makedirs(val_labels_subdir, exist_ok=True)
                with open(os.path.join(val_labels_subdir, file_name) + '.txt', 'w') as label_file:
                    label_file.write(str(i) + " 0.0 0.0 1.0 1.0") 

make_dataset("./louvre.tar.gz", "./louvre_dataset", 80, 20)

C:\Users\Max\AppData\Local\Temp\ipykernel_8532\1469184517.py:66: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extract(train_data, os.path.join(train_images_subdir, file_name))
C:\Users\Max\AppData\Local\Temp\ipykernel_8532\1469184517.py:77: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extract(train_data, os.path.join(val_images_subdir, file_name))
